# Project: 멋진 챗봇 만들기

## 루브릭

|평가문항|평가문항|
|:--|:--|
|1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?|챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.|
|2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?|과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.|
|3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?|주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.|

## 라이브러리 버전 확인

```python
import numpy 
import pandas 
import tensorflow 
import nltk
import gensim

print(numpy.__version__)
print(pandas.__version__)
print(tensorflow.__version__)
print(nltk.__version__)
print(gensim.__version__)
```

```
1.21.4
1.3.3
2.6.0
3.6.5
4.1.2
```

In [1]:
# !curl -s https://raw.githubusercontent.com/teddylee777/machine-learning/master/99-Misc/01-Colab/mecab-colab.sh | bash

In [2]:
# !pip install sentencepiece

# 필요한 라이브러리 import

In [3]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

import gensim
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from sklearn.model_selection import train_test_split
from tqdm import tqdm_notebook
from collections import Counter
from konlpy.tag import Mecab

# 데이터 다운로드

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv')

In [5]:
df.head()

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0


In [6]:
len(df)

11823

In [7]:
src = []
tgt = []
for s, t in zip(df['Q'], df['A']):
    src.append(str(s))
    tgt.append(str(t))

In [8]:
print(len(src))
print(len(tgt))
print()

for i in range(10):
    print('Q :', df['Q'][i])
    print('A :', df['A'][i])    
    print()

11823
11823

Q : 12시 땡!
A : 하루가 또 가네요.

Q : 1지망 학교 떨어졌어
A : 위로해 드립니다.

Q : 3박4일 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : 3박4일 정도 놀러가고 싶다
A : 여행은 언제나 좋죠.

Q : PPL 심하네
A : 눈살이 찌푸려지죠.

Q : SD카드 망가졌어
A : 다시 새로 사는 게 마음 편해요.

Q : SD카드 안돼
A : 다시 새로 사는 게 마음 편해요.

Q : SNS 맞팔 왜 안하지ㅠㅠ
A : 잘 모르고 있을 수도 있어요.

Q : SNS 시간낭비인 거 아는데 매일 하는 중
A : 시간을 정하고 해보세요.

Q : SNS 시간낭비인데 자꾸 보게됨
A : 시간을 정하고 해보세요.



# 데이터 정제 및 토큰화

In [9]:
# 결측치 확인
df.isnull().sum()

Q        0
A        0
label    0
dtype: int64

In [10]:
df.drop_duplicates(['A'], inplace=True)
len(df)

7779

In [11]:
df.drop_duplicates(['Q'], inplace=True)
len(df)

7727

In [12]:
print('total data length :', len(df))

total data length : 7727


In [13]:
mecab = Mecab()

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()  # 모든 입력을 소문자로 변환합니다.
    sentence = re.sub(r"[^0-9a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣?.!,]+", " ", sentence)  # 알파벳, 문장부호, 한글만 남기고 모두 제거합니다.
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)  # 문장부호 양옆에 공백을 추가합니다.
    sentence = re.sub(r'[" "]+', " ", sentence)  # 여러개의 공백 한개로
    sentence = sentence.strip()  # 문장 앞뒤의 불필요한 공백을 제거합니다.
    corpus = mecab.morphs(sentence)
    
    return corpus

In [14]:
questions = [preprocess_sentence(q) for q in df['Q']]
answers = [preprocess_sentence(a) for a in df['A']]

print('questions data size :', len(questions))
print('answers data size :', len(answers))
print()

for i in range(10):
    print('Q :', questions[i])
    print('A :', answers[i])    
    print()

questions data size : 7727
answers data size : 7727

Q : ['12', '시', '땡', '!']
A : ['하루', '가', '또', '가', '네요', '.']

Q : ['1', '지망', '학교', '떨어졌', '어']
A : ['위로', '해', '드립니다', '.']

Q : ['3', '박', '4', '일', '놀', '러', '가', '고', '싶', '다']
A : ['여행', '은', '언제나', '좋', '죠', '.']

Q : ['ppl', '심하', '네']
A : ['눈살', '이', '찌푸려', '지', '죠', '.']

Q : ['sd', '카드', '망가졌', '어']
A : ['다시', '새로', '사', '는', '게', '마음', '편해요', '.']

Q : ['sns', '맞', '팔', '왜', '안', '하', '지', 'ㅠㅠ']
A : ['잘', '모르', '고', '있', '을', '수', '도', '있', '어요', '.']

Q : ['sns', '시간', '낭비', '인', '거', '아', '는데', '매일', '하', '는', '중']
A : ['시간', '을', '정하', '고', '해', '보', '세요', '.']

Q : ['sns', '보', '면', '나', '만', '빼', '고', '다', '행복', '해', '보여']
A : ['자랑', '하', '는', '자리', '니까요', '.']

Q : ['가끔', '궁금', '해']
A : ['그', '사람', '도', '그럴', '거', '예요', '.']

Q : ['가끔', '은', '혼자', '인', '게', '좋', '다']
A : ['혼자', '를', '즐기', '세요', '.']



In [15]:
cleaned_corpus = list(set(zip(src,tgt)))

In [16]:
src_corpus = []
tgt_corpus = []

for tmp in cleaned_corpus:
    tmp_src = preprocess_sentence(tmp[0])
    tmp_tgt = preprocess_sentence(tmp[1])
    if len(tmp) <= 40:
        src_corpus.append(tmp_src)
        tgt_corpus.append(tmp_tgt)

que_corpus = src_corpus
ans_corpus = tgt_corpus    

print(len(src_corpus))
print(len(tgt_corpus))
print("Questions:", src_corpus[100])   
print("Answers:", tgt_corpus[100])

11750
11750
Questions: ['이', '별', '한', '달', '반', '째']
Answers: ['힘든가', '봅니다', '.']


# Augmentation

In [17]:
from gensim.models import Word2Vec

In [18]:
word2vec_path = '/content/drive/MyDrive/Project/AIFFEL/Going_Deeper/12. 번역가는 대화에도 능하다/data/ko.bin'
word2vec = Word2Vec.load(word2vec_path)

In [19]:
word2vec.similar_by_word("가족")

[('친지', 0.6574771404266357),
 ('유족', 0.6132265329360962),
 ('부부', 0.5933582782745361),
 ('부모님', 0.5929263234138489),
 ('부모', 0.5834078192710876),
 ('식구', 0.5739837288856506),
 ('친족', 0.5737326741218567),
 ('노인', 0.5734318494796753),
 ('자녀', 0.5686265230178833),
 ('부유층', 0.5621210336685181)]

In [20]:
# Lexical Substitution 구현하기

def lexical_sub(sentence, word2vec):
    res = ""
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:  # 단어장에 없는 단어
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [21]:
new_que_corpus = []
new_ans_corpus = []

# Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록 한다.
for idx in tqdm_notebook(range(len(que_corpus))):
    que_augmented = lexical_sub(que_corpus[idx], word2vec)
    ans = ans_corpus[idx]
    
    if que_augmented is not None:
        new_que_corpus.append(que_augmented.split())
        new_ans_corpus.append(ans)
        
    else:
       
        continue
    
for idx in tqdm_notebook(range(len(ans_corpus))):
    que = que_corpus[idx]
    ans_augmented = lexical_sub(ans_corpus[idx], word2vec)
    
    if ans_augmented is not None:
        new_que_corpus.append(que)
        new_ans_corpus.append(ans_augmented.split())
       
    else:
       
        continue

  0%|          | 0/11750 [00:00<?, ?it/s]

  0%|          | 0/11750 [00:00<?, ?it/s]

In [22]:
print(new_que_corpus[:10])
print(new_ans_corpus[:10])
print()
print(len(new_que_corpus))
print(len(new_ans_corpus))

[['연봉', '대응', '근무', '시간', '이', '너무', '긴', '거', '같', '애'], ['돈', '은', '어떻게', '모으', '는', '것', '임'], ['뒤', '에서', '발자국', '소리', '나', '서', '살', '램'], ['성행위', '상담', '가능', '?'], ['그냥', '궁금', '해의'], ['유부남', '됐', '어서', '!'], ['뭔가', '불공평', '해의'], ['아무리', '생각', '안', '하', '려고', '해도'], ['엄마', '랑', '여행', '안', '놀드', '봤', '는데'], ['헤어졌', '는데', '왜', '제', '번호', '을', '안', '지워', '을', '까', '이해', '해', '안', '돼', '네']]
[['야근', '수당', '을', '챙겨', '보', '세요', '.'], ['한', '푼', '두', '푼', '차곡차곡'], ['항상', '조심', '하', '세요', '.'], ['부족', '하', '지만', '열심히', '해', '드릴게요', '.'], ['궁금', '할', '수', '있', '어요', '.'], ['축하', '해요', '!'], ['세상', '은', '누구', '에게', '나', '불', '공평', '해요', '.'], ['눈', '앞', '에', '선연', '하', '기', '도', '해요', '.'], ['좋', '은', '여행', '이', '될', '거', '예요', '.'], ['바빠서', '그럴', '수', '도', '있', '어요', '.', '신경', '쓰', '지', '않', '는', '연습', '을', '해', '봐요', '.']]

20309
20309


# 데이터 벡터화

In [23]:
tgt_corpus = []

for corpus in ans_corpus:
    tgt_corpus.append(["<start>"] + corpus + ["<end>"])
    
print(tgt_corpus[0])
ans_corpus = tgt_corpus
print(ans_corpus[0])

['<start>', '야근', '수당', '을', '챙겨', '보', '세요', '.', '<end>']
['<start>', '야근', '수당', '을', '챙겨', '보', '세요', '.', '<end>']


In [24]:
voc_data = que_corpus + ans_corpus

words = np.concatenate(voc_data).tolist()
counter = Counter(words)
counter = counter.most_common(30000-2)
vocab = ['<pad>', '<unk>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
index_to_word = {index:word for word, index in word_to_index.items()}

In [25]:
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index[word] if word in word_to_index else word_to_index['<unk>'] for word in sentence]

def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<unk>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

def vectorize(corpus, word_to_index):
    data = []
    for sen in corpus:
        sen = get_encoded_sentence(sen, word_to_index)
        data.append(sen)
    return data

que_train = vectorize(que_corpus, word_to_index)
ans_train = vectorize(ans_corpus, word_to_index)

print(len(que_train))
print(len(ans_train))

11750
11750


In [26]:
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(que_train, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(ans_train, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01) # test set은 1%만

print(len(enc_train))
print(len(enc_val)) 
print(len(dec_train))
print(len(dec_val))

11632
118
11632
118


# 훈련하기

In [27]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # 인덱스가 짝수
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # 인덱스가 홀수

             
    return sinusoid_table

In [28]:
# Mask 생성
def generate_padding_mask(seq):
    # tf.math.equal: seq의 원소가 0이 되면 true로 반환, 아니면 false 반환
    # tf.cast: true를 float32로 변환
    
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # np.newaxis: numpy array의 차원 늘려주기

def generate_causality_mask(src_len, tgt_len):
    # np.cumsum(): 배열에서 행에 따라 누적되는 원소들의 누적합 계산
    # np.eye(): 대각선이 1인 seq_len x seq_len 크기의 대각행렬 생성
    
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)  # tf.cast: mask(텐서)를 float32로 변환


def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

In [29]:
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads
        
        # Linear Layer
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)  # Scaled QK

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)  # Attention Weights
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]  # batch size
        # reshape - shape의 한 원소만 -1, 의미는 전체 크기가 일정하게 유지되도록 해당 차원의 길이가 자동으로 계산
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])  # perm은 치환하는 위치를 알려줌

        return split_x

    
    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        # Linear 레이어 추가 - embedding 매핑
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [30]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out

In [31]:
# Encoder Layer 구현
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn

In [32]:
# Decoder Layer 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

In [33]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

In [34]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

In [35]:
# Transformer 전체 모델 조립
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,  # 레이어의 차원수
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
    
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
  
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        # np.newaxis: numpy array의 차원 늘려주기
        
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

In [36]:
#  Transformer 인스턴스 생성
transformer = Transformer(
    n_layers=1,
    d_model=368,
    n_heads=8,
    d_ff=1024,
    src_vocab_size=50000,
    tgt_vocab_size=50000,
    pos_len=42,
    dropout=0.2,
    shared_fc=True,
    shared_emb=True)

d_model = 368

In [37]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=1000):
        super(LearningRateScheduler, self).__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [38]:
# Learning Rate 인스턴스 선언
learning_rate = LearningRateScheduler(d_model)

# Optimizer 구현
optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [39]:
# Loss Function 정의
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [40]:
# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [41]:
# 훈련하기
def evaluate(sentence, model):
    # sentence 전처리(enc_train과 같은 모양으로)
    sentence = preprocess_sentence(sentence)
    pieces = sentence
    tokens = get_encoded_sentence(pieces, word_to_index)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    
    output = tf.expand_dims([word_to_index["<start>"]], 0) 
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
        
        # 예측 단어가 종료 토큰일 경우
        if word_to_index["<end>"] == predicted_id:
            result = get_decoded_sentence(ids, index_to_word)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns
        ##word_to_index
        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = get_decoded_sentence(ids, index_to_word)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model)
    
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    return result

In [42]:
BATCH_SIZE = 64
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

In [43]:
examples = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

In [44]:
for example in examples:
    translate(example, transformer)

Input: 지루하다, 놀러가고 싶어.
Predicted translation: 마다 다르 겠 죠 .
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 쉬 는 시간 에 는 시간 으니까요 .
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 했 길 바라 요 .
Input: 집에 있는다는 소리야.
Predicted translation: 고 싶 지 않 죠 .


# 성능 측정하기

In [45]:
# 아래 두 문장을 바꿔가며 테스트 해보세요
reference = "많 은 자연어 처리 연구자 들 이 트랜스포머 를 선호 한다".split()
candidate = "적 은 자연어 학 개발자 들 가 트랜스포머 을 선호 한다 요".split()

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

원문: ['많', '은', '자연어', '처리', '연구자', '들', '이', '트랜스포머', '를', '선호', '한다']
번역문: ['적', '은', '자연어', '학', '개발자', '들', '가', '트랜스포머', '을', '선호', '한다', '요']
BLEU Score: 0.5491004867761125


In [46]:
def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

BLEU-1: 0.5
BLEU-2: 0.18181818181818182
BLEU-3: 0.010000000000000004
BLEU-4: 0.011111111111111112

BLEU-Total: 0.05637560315259291


In [47]:
def eval_bleu(src_corpus, tgt_corpus, verbose=True):
    total_score = 0.0
    sample_size = len(tgt_corpus)

    for idx in tqdm_notebook(range(sample_size)):
        src_tokens = src_corpus[idx]
        tgt_tokens = tgt_corpus[idx]
        
        src = []
        tgt = []
        
        for word in src_tokens:
            if word !=0 and word !=1 and word !=3 and word !=4:
                src.append(word)
        
        for word in tgt_tokens:
            if word != 0 and word != 3 and word !=4:
                tgt.append(word)

        src_sentence = get_decoded_sentence(src, index_to_word)
        tgt_sentence = get_decoded_sentence(tgt, index_to_word)
        
        
        reference = preprocess_sentence(tgt_sentence)
        candidate = translate(src_sentence, transformer)

        score = sentence_bleu([reference], candidate,
                              smoothing_function=SmoothingFunction().method1)
        total_score += score

        if verbose:
            print("Source Sentence: ", src_sentence)
            print("Model Prediction: ", candidate)
            print("Real: ", reference)
            print("Score: %lf\n" % score)

    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)

In [48]:
eval_bleu(enc_val[::5], dec_val[::5], verbose=True)

  0%|          | 0/24 [00:00<?, ?it/s]

Input: 있 고 싶 다
Predicted translation: 그러 고 싶 을 자책 하 지 마세요 .
Source Sentence:  있 고 싶 다
Model Prediction:  그러 고 싶 을 자책 하 지 마세요 .
Real:  ['만', '있', '지', '마세요', '.']
Score: 0.010863

Input: 사람 은 잘 만 하 는 연애
Predicted translation: 먹 고 있 어요 .
Source Sentence:  사람 은 잘 만 하 는 연애
Model Prediction:  먹 고 있 어요 .
Real:  ['연애', '도', '연애', '에', '요', '.']
Score: 0.025099

Input: 해 보 는 게 좋 을까
Predicted translation: 는 일 이 없 어요 .
Source Sentence:  해 보 는 게 좋 을까
Model Prediction:  는 일 이 없 어요 .
Real:  ['하', '는', '편', '이', '좋', '을', '것', '같', '아요', '.']
Score: 0.022417

Input: 잘 하 고 있 는 건지 모르 겠 어
Predicted translation: 걸 아무것 도 잘 모르 겠 어요 .
Source Sentence:  잘 하 고 있 는 건지 모르 겠 어
Model Prediction:  걸 아무것 도 잘 모르 겠 어요 .
Real:  ['하', '고', '있', '을', '거', '예요', '.']
Score: 0.010182

Input: 친구 가 너무 국밥 류 만 좋 아 해 .
Predicted translation: 전환 을 해 보 세요 .
Source Sentence:  친구 가 너무 국밥 류 만 좋 아 해 .
Model Prediction:  전환 을 해 보 세요 .
Real:  ['하', '게', '먹', '자고', '해', '보', '세요', '.']
Score: 0.020448

Input: 인데 짜장면 같이 먹 을 사람 이 없 어
Pr

# 회고

- Transformer 하이퍼파라미터 조정해보기 + Epoch 추가
- 데이터 증강 더 해보기 